In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.simplefilter('ignore')

In [23]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# setting format for readability
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
combine = pd.read_csv('2000-2020_nfl_combine_data.csv', index_col = 0)
combine.head()

,Link,PlayerName,Position,School,CollegeStats,Height,Weight,40yd,Vertical,Bench,BroadJump,3Cone,Shuttle,Drafted,CombineYear,Draft Team,Round,Pick,Draft Year
0,https://www.pro-football-reference.com/draft/2...,John Abraham,OLB,South Carolina,/td,6-4,252,4.55,NaN,NaN,NaN,NaN,NaN,New York Jets / 1st / 13th pick / 2000,2000,New York Jets,1st,13th pick,2000
1,https://www.pro-football-reference.com/draft/2...,Shaun Alexander,RB,Alabama,https://www.sports-reference.com/cfb/players/s...,6-0,218,4.58,NaN,NaN,NaN,NaN,NaN,Seattle Seahawks / 1st / 19th pick / 2000,2000,Seattle Seahawks,1st,19th pick,2000
2,https://www.pro-football-reference.com/draft/2...,Darnell Alford,OT,Boston Col.,/td,6-4,334,5.56,25.0,23.0,94.0,8.48,4.98,Kansas City Chiefs / 6th / 188th pick / 2000,2000,Kansas City Chiefs,6th,188th pick,2000
3,https://www.pro-football-reference.com/draft/2...,Kyle Allamon,TE,Texas Tech,/td,6-2,253,4.97,29.0,NaN,104.0,7.29,4.49,NaN,2000,NaN,Not Drafted,Not Drafted,Not Drafted
4,https://www.pro-football-reference.com/draft/2...,Rashard Anderson,CB,Jackson State,/td,6-2,206,4.55,34.0,NaN,123.0,7.18,4.15,Carolina Panthers / 1st / 23rd pick / 2000,2000,Carolina Panthers,1st,23rd pick,2000


In [7]:
collegestats = pd.read_csv('college_stats_final.csv', index_col = 0)
collegestats.head()

,Link,Defense_Games,Solo_Tackles,Assisted_Tackles,Ttl_Tackles,Loss,Sacks,Defensive_Interceptions,Def_Int_Yds,Yds_per_Int,...,Kickoff_Returns,KO_Return_Yds,Yds_per_KO_Return,KO_Returned_for_TD,Scoring_Games,TD_Other,Ttl_TDs,2PT_Converstions,Safety,TTL_Points
0,https://www.sports-reference.com/cfb/players/s...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11.0,NaN,24.0,NaN,NaN,144
1,https://www.sports-reference.com/cfb/players/l...,12.0,NaN,NaN,NaN,NaN,NaN,1.0,27.0,27.0,...,0.0,0.0,0.0,0.0,12.0,NaN,0.0,NaN,NaN,0
2,https://www.sports-reference.com/cfb/players/j...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,https://www.sports-reference.com/cfb/players/a...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11.0,NaN,5.0,NaN,NaN,30
4,https://www.sports-reference.com/cfb/players/t...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11.0,NaN,1.0,NaN,NaN,6


In [14]:
collegestats.rename({'Link': 'CollegeStats'}, axis=1, inplace=True)
collegestats.head()

,CollegeStats,Defense_Games,Solo_Tackles,Assisted_Tackles,Ttl_Tackles,Loss,Sacks,Defensive_Interceptions,Def_Int_Yds,Yds_per_Int,...,Kickoff_Returns,KO_Return_Yds,Yds_per_KO_Return,KO_Returned_for_TD,Scoring_Games,TD_Other,Ttl_TDs,2PT_Converstions,Safety,TTL_Points
0,https://www.sports-reference.com/cfb/players/s...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11.0,NaN,24.0,NaN,NaN,144
1,https://www.sports-reference.com/cfb/players/l...,12.0,NaN,NaN,NaN,NaN,NaN,1.0,27.0,27.0,...,0.0,0.0,0.0,0.0,12.0,NaN,0.0,NaN,NaN,0
2,https://www.sports-reference.com/cfb/players/j...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,https://www.sports-reference.com/cfb/players/a...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11.0,NaN,5.0,NaN,NaN,30
4,https://www.sports-reference.com/cfb/players/t...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11.0,NaN,1.0,NaN,NaN,6


In [25]:
df = pd.merge(combine, collegestats, how='left', left_on='CollegeStats', right_on='CollegeStats', )
print(df.shape)
df.head()
# pd.merge(product,customer,left_on='Product_name',right_on='Purchased_Product')

(6899, 83)


,Link,PlayerName,Position,School,CollegeStats,Height,Weight,40yd,Vertical,Bench,BroadJump,3Cone,Shuttle,Drafted,CombineYear,Draft Team,Round,Pick,Draft Year,Defense_Games,Solo_Tackles,Assisted_Tackles,Ttl_Tackles,Loss,Sacks,Defensive_Interceptions,Def_Int_Yds,Yds_per_Int,Pick_6,Defended_Passes,Recovered_Fumbles,Rec_Fumbles_Yds,Fumbles_Returned_TD,Forced_Fumbles,Passing_Games,Completions,Pass_Attempts,Completion_Percent,Pass_Yards,Pass_Yds_per_Attempt,Adj_Pass_Yds_per_Attempt,Pass_TDs,Pass_Interceptions,Passer_Rating,Rushing/Receiving_Games,Rush_Attempts,Rush_Yds,Rush_Yds_per_Attempt,Rush_TDs,Receptions,Rec_Yds,Rec_Yds_per_Reception,Rec_TDs,Plays_from_Scrimmage,Scrimmage_Yds,Scrimmage_Yds_per_Attempt,Scrimmage_TDs,Kicking_Games,XP_Made,XP_Attempts,XP_Percent,FG_Made,FG_Attempts,FG_Percent,TTL_Kicking_Points,#Punts,Punt_Yds,Yds_per_Punt,PuntRet_Games,Punt_Returns,Punt_Return_Yds,Yrds_per_Return,Punt_Returned_for_TD,Kickoff_Returns,KO_Return_Yds,Yds_per_KO_Return,KO_Returned_for_TD,Scoring_Games,TD_Other,Ttl_TDs,2PT_Converstions,Safety,TTL_Points
0,https://www.pro-football-reference.com/draft/2...,John Abraham,OLB,South Carolina,/td,6-4,252,4.55,nan,nan,nan,nan,nan,New York Jets / 1st / 13th pick / 2000,2000,New York Jets,1st,13th pick,2000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,https://www.pro-football-reference.com/draft/2...,Shaun Alexander,RB,Alabama,https://www.sports-reference.com/cfb/players/s...,6-0,218,4.58,nan,nan,nan,nan,nan,Seattle Seahawks / 1st / 19th pick / 2000,2000,Seattle Seahawks,1st,19th pick,2000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,302.00,1383.00,4.60,19.00,25.00,323.00,12.90,4.00,327.00,1706.00,5.20,23.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,nan,24.00,nan,nan,144.00
2,https://www.pro-football-reference.com/draft/2...,Darnell Alford,OT,Boston Col.,/td,6-4,334,5.56,25.00,23.00,94.00,8.48,4.98,Kansas City Chiefs / 6th / 188th pick / 2000,2000,Kansas City Chiefs,6th,188th pick,2000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,https://www.pro-football-reference.com/draft/2...,Kyle Allamon,TE,Texas Tech,/td,6-2,253,4.97,29.00,nan,104.00,7.29,4.49,NaN,2000,NaN,Not Drafted,Not Drafted,Not Drafted,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,https://www.pro-football-reference.com/draft/2...,Rashard Anderson,CB,Jackson State,/td,6-2,206,4.55,34.00,nan,123.00,7.18,4.15,Carolina Panthers / 1st / 23rd pick / 2000,2000,Carolina Panthers,1st,23rd pick,2000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [27]:
df.to_pickle('combined_data')

In [28]:
df.isnull().sum()

Link                            0
PlayerName                      0
Position                        0
School                          0
CollegeStats                    0
Height                          0
Weight                          0
40yd                          317
Vertical                     1582
Bench                        2209
BroadJump                    1635
3Cone                        2545
Shuttle                      2458
Drafted                      2418
CombineYear                     0
Draft Team                   2418
Round                           0
Pick                            0
Draft Year                      0
Defense_Games                1649
Solo_Tackles                 2196
Assisted_Tackles             2196
Ttl_Tackles                  2196
Loss                         2196
Sacks                        2196
Defensive_Interceptions      1888
Def_Int_Yds                  1888
Yds_per_Int                  4302
Pick_6                       1889
Defended_Passe